# Connectivity

## PLI — Phase-Lag Index

In [ ]:
import mne
import numpy as np
import os
from mne_connectivity import spectral_connectivity_epochs

# 1. Setup
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJ = "sub-001"
SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
DATA_DIR = os.path.join(SUBJ_DIR, "data")
EPO_FILE = os.path.join(SUBJ_DIR, "epo_001_raw.fif")

# Load epochs
epochs = mne.read_epochs(EPO_FILE, preload=True)
sfreq = epochs.info['sfreq']

# Define bands for connectivity (standard canonical bands)
con_bands = {
    'delta': (1, 4), 
    'theta': (4, 8), 
    'alpha': (8, 12), 
    'beta': (12, 30)
}

# 2. Compute Connectivity (PLI)
results = {}
for band_name, (fmin, fmax) in con_bands.items():
    print(f"Computing {band_name} connectivity...")
    con_obj = spectral_connectivity_epochs(
        epochs,
        method='pli',          # or 'plv' for PLV
        mode='fourier',
        sfreq=sfreq,
        fmin=fmin,
        fmax=fmax,
        faverage=True,
        mt_adaptive=False,
        n_jobs=1
    )
    # Extract the actual connectivity matrix
    con_mat = con_obj.get_data(output='dense')  # shape: n_channels x n_channels
    results[band_name] = con_mat


# 3. Save results in subject's data folder
os.makedirs(DATA_DIR, exist_ok=True)

for band_name, con_mat in results.items():
    out_file = os.path.join(DATA_DIR, f"{SUBJ}_{band_name}_pli.npy")
    np.save(out_file, con_mat)
    print(f"Saved {band_name} PLI matrix to {out_file}")


In [10]:
import mne
import numpy as np
import os
from mne_connectivity import spectral_connectivity_epochs

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1,150)]

CON_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
}

for SUBJ in SUBJECTS:
    print(f"Processing {SUBJ}...")

    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    DATA_DIR = os.path.join(SUBJ_DIR, "data")
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{SUBJ[-3:]}_raw.fif")

    if not os.path.exists(EPO_FILE):
        print("  Missing epochs file, skipping")
        continue

    os.makedirs(DATA_DIR, exist_ok=True)

    epochs = mne.read_epochs(EPO_FILE, preload=True)
    sfreq = epochs.info["sfreq"]
    n_ch = len(epochs.ch_names)

    for band_name, (fmin, fmax) in CON_BANDS.items():
        print(f"  Computing {band_name} PLV...")

        con = spectral_connectivity_epochs(
            epochs,
            method="pli",
            mode="fourier",
            sfreq=sfreq,
            fmin=fmin,
            fmax=fmax,
            faverage=True,
            n_jobs=1,
            verbose=False
        )

        # Dense square matrix
        mat = con.get_data(output="dense")  # (n_ch, n_ch, 1)
        mat = mat[:, :, 0]

        # HARD ASSERT
        assert mat.shape == (n_ch, n_ch), f"{SUBJ} {band_name} bad shape {mat.shape}"

        out_file = os.path.join(DATA_DIR, f"{SUBJ}_{band_name}_pli_matrix.npy")
        np.save(out_file, mat)

        print(f"    Saved {band_name} PLV: {mat.shape}")


Processing sub-001...
Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
77 matching events found
No baseline correction applied
0 projection items activated
  Computing delta PLV...


C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:29: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)
C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:36: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 77 events (all good), 0 – 4 s (baseline off), ~35.4 MiB, data loaded,
 '1': 77>, so metadata was not modified.
  con = spectral_connectivity_epochs(
C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:36: RuntimeWarning: fmin=1.000 Hz corresponds to 4.004 < 5 cycles based on the epoch length 4.004 sec, need at least 5.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    Saved delta PLV: (60, 60)
  Computing theta PLV...
    Saved theta PLV: (60, 60)
  Computing alpha PLV...


C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:36: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 77 events (all good), 0 – 4 s (baseline off), ~35.4 MiB, data loaded,
 '1': 77>, so metadata was not modified.
  con = spectral_connectivity_epochs(
C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:36: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 77 events (all good), 0 – 4 s (baseline off), ~35.4 MiB, data loaded,
 '1': 77>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    Saved alpha PLV: (60, 60)
  Computing beta PLV...


...

    Saved theta PLV: (60, 60)
  Computing alpha PLV...


C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:36: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 63 events (all good), 0 – 4 s (baseline off), ~28.9 MiB, data loaded,
 '1': 63>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    Saved alpha PLV: (60, 60)
  Computing beta PLV...


C:\Users\User\AppData\Local\Temp\ipykernel_21420\1604758086.py:36: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 63 events (all good), 0 – 4 s (baseline off), ~28.9 MiB, data loaded,
 '1': 63>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    Saved beta PLV: (60, 60)
